In [44]:
import tifffile as tif
from glob import glob
from collections import OrderedDict
import nrrd
import os
import matplotlib.pyplot as plt
import cv2
import re
import numpy as np
from octvision3d.utils import get_filenames, bgr2rgb, natural_sort

DEBUG = True

In [45]:
# def get_filenames(path, ext):
#     return sorted(glob(f"{path}/*.{ext}"))

# def bgr2rgb(image):
#     return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
#     return [int(text) if text.isdigit() else text.lower()
#             for text in _nsre.split(s)]

In [46]:
dir_path = "/data/dkermany_data/OCT/phase-5/NORMAL-10"

In [47]:
vol_paths = [f for f in get_filenames(dir_path, "tif") if "slo" not in f]
seg_paths = [f for f in get_filenames(dir_path, "seg.nrrd")]
assert len(vol_paths) == len(seg_paths)
print(vol_paths)

['/data/dkermany_data/OCT/phase-5/NORMAL-10/RAI02_2-0279939-1XX-X-XXX-0-01-00000-9-3D.tif', '/data/dkermany_data/OCT/phase-5/NORMAL-10/RAI02_2-0279939-1XX-X-XXX-0-11-00000-9-3D.tif', '/data/dkermany_data/OCT/phase-5/NORMAL-10/RAI02_2-0279939-1XX-X-XXX-0-44-00000-9-3D.tif', '/data/dkermany_data/OCT/phase-5/NORMAL-10/RAI02_2-0279939-1XX-X-XXX-0-54-00000-9-3D.tif', '/data/dkermany_data/OCT/phase-5/NORMAL-10/RAI02_2-0279939-1XX-X-XXX-0-59-00000-9-3D.tif']


In [48]:
def overlay_segments(segments, colors):
    """
    Overlay binary masks onto a blank image with specified colors.

    :param masks: List of binary masks (numpy arrays).
    :param colors: List of colors corresponding to each mask.
    :return: Image with masks overlaid.
    """
    # Create a blank image

    final_images = np.zeros(segments.T.shape[:-1] + (3,), dtype=np.uint8)

    for segment, color in zip(segments, colors):
        for i, slice in enumerate(segment.T):
            bgr_image = cv2.cvtColor(slice, cv2.COLOR_GRAY2BGR)
            final_images[i] += bgr_image * color

    return final_images

In [49]:
def check_unlabeled_pixels(bitmap):
    print(bitmap.shape)
    for segment in bitmap:
        print(segment.T.shape)

In [52]:
pattern = re.compile("^Segment\d+")

for vol_path, seg_path in zip(vol_paths, seg_paths):

    vol = tif.imread(vol_path)
    bitmap, header = nrrd.read(seg_path)
    segment_colors = {k.split("_")[0]: v for k, v in header.items() if k.endswith("Color")}
    segment_names = {k.split("_")[0]: v for k, v in header.items() if k.endswith("Name")}
    sorted_color_map = sorted(segment_colors.items(), key=lambda x: natural_sort_key(x[0]))
    sorted_color_map = natural_sort(segment_colors.items())
    _, sorted_colors = list(zip(*sorted_color_map))
    rgb_colors = np.array([[round(255.*float(c)) for c in i.split(" ")] for i in sorted_colors], dtype=np.uint8)

    if DEBUG:
        print(f"Name: {os.path.splitext(os.path.basename(vol_path))[0]}")
        print(f"TIFF type: {type(vol)}, TIFF shape: {vol.shape}, SEG.NRRD type: {type(bitmap)}, SEG.NRRD shape: {bitmap.shape}")
        print(f"segment_names: {segment_names}")
        print(f"segment_colors: {segment_colors}")
        print(f"sorted_color_map: {sorted_color_map}")
        print(f"sorted_colors: {sorted_colors}")
        print(f"rgb_colors: {rgb_colors}")


    # seg_vol = overlay_segments(bitmap, rgb_colors)
    check_unlabeled_pixels(bitmap)

    # for seg_slice, vol_slice in zip(seg_vol, vol):
    #     f, ax = plt.subplots(1,2, figsize=(25,14))
    #     ax[0].imshow(seg_slice)
    #     ax[1].imshow(vol_slice, cmap="gray")
    break

    

Name: RAI02_2-0279939-1XX-X-XXX-0-01-00000-9-3D
TIFF type: <class 'numpy.ndarray'>, TIFF shape: (19, 496, 1024), SEG.NRRD type: <class 'numpy.ndarray'>, SEG.NRRD shape: (14, 1024, 496, 19)
segment_names: {'Segment0': 'CNV', 'Segment10': 'HYA', 'Segment11': 'SHS', 'Segment12': 'ART', 'Segment13': 'ERM', 'Segment1': 'DRU', 'Segment2': 'EX', 'Segment3': 'FLU', 'Segment4': 'GA', 'Segment5': 'HEM', 'Segment6': 'RPE', 'Segment7': 'RET', 'Segment8': 'CHO', 'Segment9': 'VIT'}
segment_colors: {'Segment0': '0.501961 0.682353 0.501961', 'Segment10': '0.46 0.98 0.99', 'Segment11': '0.69 0.99 0.82', 'Segment12': '0.99 0.99 0.33', 'Segment13': '0.22 0.49 0.97', 'Segment1': '0.945098 0.839216 0.568627', 'Segment2': '0.694118 0.478431 0.396078', 'Segment3': '0.435294 0.721569 0.823529', 'Segment4': '0.847059 0.396078 0.309804', 'Segment5': '0.866667 0.509804 0.396078', 'Segment6': '0.564706 0.933333 0.564706', 'Segment7': '0.635 0 1', 'Segment8': '0.56 0.56 0.44', 'Segment9': '0.88 0.94 0.99'}
sorted_